In [1]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

In [2]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD_adv.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA_adv.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD_adv',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA_adv',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP_adv',
    'ALL': '/workspace/workspace/ufo-prediction/image_data'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=1500000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Enter country code (FRA, NLD, ESP) or ALL: ESP
Directory /workspace/workspace/ufo-prediction/image_data_ESP_adv already exists.
Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
             lon        lat  age_right                         id
0      -1.798480  40.972110     1940.0    v0.1-ESP.4.4.1.53_1-112
1      -4.128549  37.172721     1960.0   v0.1-ESP.1.4.5.29_1-9551
2       2.608557  39.594611     2000.0  v0.1-ESP.13.1.8.1_1-40343
3      -2.020573  40.296965     1900.0    v0.1-ESP.4.3.2.12_1-371
4       2.496946  39.511195     2002.0  v0.1-ESP.13.1.7.4_1-12584
...          ...        ...        ...                        ...
894009 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
894010  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
894011 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
894012 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
894013 -4.000000        NaN        NaN         

In [3]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_ESP_adv
             lon        lat  age_right                         id
0      -1.798480  40.972110     1940.0    v0.1-ESP.4.4.1.53_1-112
1      -4.128549  37.172721     1960.0   v0.1-ESP.1.4.5.29_1-9551
2       2.608557  39.594611     2000.0  v0.1-ESP.13.1.8.1_1-40343
3      -2.020573  40.296965     1900.0    v0.1-ESP.4.3.2.12_1-371
4       2.496946  39.511195     2002.0  v0.1-ESP.13.1.7.4_1-12584
...          ...        ...        ...                        ...
894009 -5.984130  37.675687     2011.0    v0.1-ESP.1.8.1.4_1-5558
894010  2.505601  41.879092     1900.0    v0.1-ESP.6.2.6.19_1-823
894011 -3.924419  37.463344     1980.0   v0.1-ESP.1.6.9.1_1-10785
894012 -0.220692  38.513329     1945.0  v0.1-ESP.10.1.7.17_1-5766
894013 -4.000000        NaN        NaN                        NaN

[894014 rows x 4 columns]


In [ ]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

#Initialise a counter for the loop iterations
iteration_counter = 0

for index, row in kartaview_keys.iterrows():
    iteration_counter += 1 # Increment the counter with each iteration
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Only save the updated DataFrame to a CSV file every 100th instance
    if iteration_counter % 100 == 0:
        print(f"Saving progress at iteration {iteration_counter} to {processed_df_path}. ")
        print("Number of buildings remaining: ",len(kartaview_keys))
        kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))
        
if iteration_counter % 100 != 0:
    print(f"Saving final progress")
    print("Number of buildings remaining: ",len(kartaview_keys))
    kartaview_keys.to_csv(processed_df_path, index=False)

Images will be saved in: /workspace/workspace/ufo-prediction/image_data_ESP_adv
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
Inconsistency detected between the image directory and the kartaview_keys path. Please check.
Number of buildings remaining:  894014
No suitable images found for location: lon=-1.798480, lat=40.972110
No suitable images found for location: lon=-1.79848, lat=40.97211
No suitable images found for location: lon=-1.7985, lat=40.9721
No suitable images found for location: lon=-1.798, lat=40.972
Unable to retrieve data from the API with sufficient precision for location: lon=-1.798480170184035, lat=40.97210984198054
No suitable images found for location: lon=-4.128549, lat=37.172721
No suitable images found for location: lon=-4.12855, lat=37.17272
No suitable images found for location: lon=-4.1285, lat=37.1727
No suitable images found for location: lon=-4.129, lat=37.173
Unable to retrieve data from the API with sufficie

No suitable images found for location: lon=-2.241320, lat=40.450428
No suitable images found for location: lon=-2.24132, lat=40.45043
No suitable images found for location: lon=-2.2413, lat=40.4504
No suitable images found for location: lon=-2.241, lat=40.450
Unable to retrieve data from the API with sufficient precision for location: lon=-2.241320219983023, lat=40.45042794222531
No suitable images found for location: lon=-5.987302, lat=37.675862
No suitable images found for location: lon=-5.98730, lat=37.67586
No suitable images found for location: lon=-5.9873, lat=37.6759
No suitable images found for location: lon=-5.987, lat=37.676
Unable to retrieve data from the API with sufficient precision for location: lon=-5.9873019711345545, lat=37.67586184863664
No suitable images found for location: lon=-1.269283, lat=38.074066
No suitable images found for location: lon=-1.26928, lat=38.07407
No suitable images found for location: lon=-1.2693, lat=38.0741
No suitable images found for locati

No suitable images found for location: lon=2.064, lat=41.497
Unable to retrieve data from the API with sufficient precision for location: lon=2.0642051191492072, lat=41.496637274673006
No suitable images found for location: lon=-1.860682, lat=38.989526
No suitable images found for location: lon=-1.86068, lat=38.98953
No suitable images found for location: lon=-1.8607, lat=38.9895
No suitable images found for location: lon=-1.861, lat=38.990
Unable to retrieve data from the API with sufficient precision for location: lon=-1.8606817559024893, lat=38.98952640842225
No suitable images found for location: lon=-0.442913, lat=38.710331
No suitable images found for location: lon=-0.44291, lat=38.71033
No suitable images found for location: lon=-0.4429, lat=38.7103
No suitable images found for location: lon=-0.443, lat=38.710
Unable to retrieve data from the API with sufficient precision for location: lon=-0.4429134852676199, lat=38.71033110227857
No suitable images found for location: lon=3.07

No suitable images found for location: lon=-8.684471, lat=42.221211
No suitable images found for location: lon=-8.68447, lat=42.22121
No suitable images found for location: lon=-8.6845, lat=42.2212
No suitable images found for location: lon=-8.684, lat=42.221
Unable to retrieve data from the API with sufficient precision for location: lon=-8.684471232862949, lat=42.22121057253614
No suitable images found for location: lon=-4.005886, lat=41.153093
No suitable images found for location: lon=-4.00589, lat=41.15309
No suitable images found for location: lon=-4.0059, lat=41.1531
No suitable images found for location: lon=-4.006, lat=41.153
Unable to retrieve data from the API with sufficient precision for location: lon=-4.005885595527532, lat=41.15309262077249
No suitable images found for location: lon=-0.384583, lat=39.732951
No suitable images found for location: lon=-0.38458, lat=39.73295
No suitable images found for location: lon=-0.3846, lat=39.7330
No suitable images found for locatio

No suitable images found for location: lon=-5.357, lat=40.152
Unable to retrieve data from the API with sufficient precision for location: lon=-5.357134700660822, lat=40.15151774914527
No suitable images found for location: lon=-4.643802, lat=39.716061
No suitable images found for location: lon=-4.64380, lat=39.71606
No suitable images found for location: lon=-4.6438, lat=39.7161
No suitable images found for location: lon=-4.644, lat=39.716
Unable to retrieve data from the API with sufficient precision for location: lon=-4.643802457492077, lat=39.71606146606128
No suitable images found for location: lon=-9.032293, lat=42.667697
No suitable images found for location: lon=-9.03229, lat=42.66770
No suitable images found for location: lon=-9.0323, lat=42.6677
No suitable images found for location: lon=-9.032, lat=42.668
Unable to retrieve data from the API with sufficient precision for location: lon=-9.03229307248667, lat=42.66769702622808
No suitable images found for location: lon=-4.3953

...

No suitable images found for location: lon=-5.66828, lat=38.92228
No suitable images found for location: lon=-5.6683, lat=38.9223
No suitable images found for location: lon=-5.668, lat=38.922
Unable to retrieve data from the API with sufficient precision for location: lon=-5.66828015085504, lat=38.92228300207371
No suitable images found for location: lon=-5.348629, lat=43.183037
No suitable images found for location: lon=-5.34863, lat=43.18304
No suitable images found for location: lon=-5.3486, lat=43.1830
No suitable images found for location: lon=-5.349, lat=43.183
Unable to retrieve data from the API with sufficient precision for location: lon=-5.348628680248869, lat=43.18303656556867
No suitable images found for location: lon=-4.912495, lat=40.243413
No suitable images found for location: lon=-4.91250, lat=40.24341
No suitable images found for location: lon=-4.9125, lat=40.2434
No suitable images found for location: lon=-4.912, lat=40.243
Unable to retrieve data from the API with s

No suitable images found for location: lon=-0.03933, lat=41.23633
No suitable images found for location: lon=-0.0393, lat=41.2363
No suitable images found for location: lon=-0.039, lat=41.236
Unable to retrieve data from the API with sufficient precision for location: lon=-0.0393254488071519, lat=41.23633179243325
No suitable images found for location: lon=-1.926843, lat=39.344728
No suitable images found for location: lon=-1.92684, lat=39.34473
No suitable images found for location: lon=-1.9268, lat=39.3447
No suitable images found for location: lon=-1.927, lat=39.345
Unable to retrieve data from the API with sufficient precision for location: lon=-1.9268430669140275, lat=39.34472820340096
No suitable images found for location: lon=-1.757471, lat=38.483828
No suitable images found for location: lon=-1.75747, lat=38.48383
No suitable images found for location: lon=-1.7575, lat=38.4838
No suitable images found for location: lon=-1.757, lat=38.484
Unable to retrieve data from the API wit

No suitable images found for location: lon=-6.209795, lat=39.636491
No suitable images found for location: lon=-6.20979, lat=39.63649
No suitable images found for location: lon=-6.2098, lat=39.6365
No suitable images found for location: lon=-6.210, lat=39.636
Unable to retrieve data from the API with sufficient precision for location: lon=-6.209794517988328, lat=39.636490552610994
No suitable images found for location: lon=2.048841, lat=41.459500
No suitable images found for location: lon=2.04884, lat=41.45950
No suitable images found for location: lon=2.0488, lat=41.4595
No suitable images found for location: lon=2.049, lat=41.460
Unable to retrieve data from the API with sufficient precision for location: lon=2.0488414404206274, lat=41.45950027634293
No suitable images found for location: lon=-3.618867, lat=39.070700
No suitable images found for location: lon=-3.61887, lat=39.07070
No suitable images found for location: lon=-3.6189, lat=39.0707
No suitable images found for location: 

No suitable images found for location: lon=-3.175, lat=37.411
Unable to retrieve data from the API with sufficient precision for location: lon=-3.174710728290972, lat=37.411294778723736
No suitable images found for location: lon=-6.623850, lat=42.771578
No suitable images found for location: lon=-6.62385, lat=42.77158
No suitable images found for location: lon=-6.6239, lat=42.7716
No suitable images found for location: lon=-6.624, lat=42.772
Unable to retrieve data from the API with sufficient precision for location: lon=-6.623850460232999, lat=42.77157831474931
No suitable images found for location: lon=-15.480385, lat=28.095868
No suitable images found for location: lon=-15.48038, lat=28.09587
No suitable images found for location: lon=-15.4804, lat=28.0959
No suitable images found for location: lon=-15.480, lat=28.096
Unable to retrieve data from the API with sufficient precision for location: lon=-15.48038469923587, lat=28.095867564431177
No suitable images found for location: lon=

No suitable images found for location: lon=0.340648, lat=42.192758
No suitable images found for location: lon=0.34065, lat=42.19276
No suitable images found for location: lon=0.3406, lat=42.1928
No suitable images found for location: lon=0.341, lat=42.193
Unable to retrieve data from the API with sufficient precision for location: lon=0.3406480737496771, lat=42.19275793277056
No suitable images found for location: lon=-5.000402, lat=41.091212
No suitable images found for location: lon=-5.00040, lat=41.09121
No suitable images found for location: lon=-5.0004, lat=41.0912
No suitable images found for location: lon=-5.000, lat=41.091
Unable to retrieve data from the API with sufficient precision for location: lon=-5.000401870625191, lat=41.091212351367965
No suitable images found for location: lon=2.886461, lat=39.621436
No suitable images found for location: lon=2.88646, lat=39.62144
No suitable images found for location: lon=2.8865, lat=39.6214
No suitable images found for location: lon